In [40]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera

from scipy.signal import find_peaks, butter, filtfilt

import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np

# Camera resolution

res = (640, 480)

CAMERA_CENTER = res[0] // 2
       
from pwm import PWM

## Setup servo and motor PWM signal to control the steering angle and the speed.

The values specified period and duty cycle ar in nanoseconds.

Servo and motor controlled follow the following protocol:
  1. Control signal is read 50 times per second by each device (servo and speed controller). Thus period is 20 ms => 20000000 ns
  2. To send the minimum value (full left for servo and break for motor) the duty cycle should be 1 ms => 1000000 ns
  3. Maximum value corresponds to a duty cycle of 2 ms => 2000000 ns
  
However, these devices are not perfect and you can set a duty cycle outside this range and still get a response. I would recoment to play with it and find are the limits in your case. Pay attention that servo is limited by the mechanical construction how much it can turn the wheels. Do not set these limits to a value which servo cannot execute as it will try to do that and eventually burn.

In [41]:
# Enable servo
SERVO_MIDDLE = 1500000

servo = PWM(1)
servo.period = 20000000
servo.duty_cycle = SERVO_MIDDLE
servo.enable = True

In [42]:
# Enable servo
MOTOR_BRAKE = 1000000

motor = PWM(0)
motor.period = 20000000
motor.duty_cycle = MOTOR_BRAKE
motor.enable = True

In [45]:
motor.duty_cycle = MOTOR_BRAKE 

## The self driving code

Quick explanation on how this algorithm works:
  1. Initialize the camera, use a frame rate of 10 to 20 fps
  2. Get the coefficients of the butterworth filter of order 3 and cut-off frequency of 0.1 (change cut of frequency if the peak functions is not working)
  3. For each frame captured from the camera:
    1. Get the blak and white part of the image
    2. Select line 195 to use for lane detection (this number depends on your camera angle)
    3. Filter the line and find the peaks that are greater than 160. White parts of the image have higher values than the black. The threshold of 160 depends on the ambient light, lower if the room is darker
    4. Find the peaks left and right of the camera center, these peaks should corresponds to the white lines delimiting the track,
    5. Compute the center point between these lines. This is where we want our car
    6. Compute the error by substracting the center point from camera middle and multiply by 5000. 5000 is our P gain.
    7. Adjust the duty cycle of the servo such that the

In [44]:
# Run a track detection algorithm on a single horizontal line.
# Uses YUV420 image format as the Y component corresponds to image intensity (gray image)
# and thus there is no need to convert from RGB to BW

RUN_TIMER = 25 # seconds
history=[]

camera = PiCamera()
        
# Check the link below for the combinations between mode and resolution
# https://picamera.readthedocs.io/en/release-1.13/fov.html#sensor-modes
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 10

# Initialize the buffer and start capturing
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

# Measure the time needed to process 300 images to estimate the FPS
t = time.time()

# To filter the noise in the image we use a 3rd order Butterworth filter

# Wn = 0.02, the cut-off frequency, acceptable values are from 0 to 1
b, a = butter(3, 0.1)

line_pos    = CAMERA_CENTER
first_frame = True

# start car
motor.duty_cycle = MOTOR_BRAKE + 120000
track_width = 0
MOTOR_MAX = 350000
MOTOR_STRAIGHT = MOTOR_MAX
SPEED_COMFY = 250000
MOTOR_TURN_INIT = 150000
MOTOR_TURN = MOTOR_TURN_INIT
TURN_COMFY = 250000
turn_ratio_start = 2
turn_ratio_end = 1.5
inc = 0.1
turn_ratio = turn_ratio_start
for f in stream:
    if first_frame:
        first_frame = False
        # Reset the buffer for the next image
        rawCapture.truncate(0)
        continue
    
    # Stop after RUN_TIMER seconds
    if (time.time() - t) > RUN_TIMER:
        break
        
    # Get the intensity component of the image (a trick to get black and white images)
    I = f.array[:, :, 0]
    
    # Reset the buffer for the next image
    rawCapture.truncate(0)
    
    # Select a horizontal line in the middle of the image
    L = I[195, :]

    # Smooth the transitions so we can detect the peaks 
    Lf = filtfilt(b, a, L)
    history.append(Lf)
    
    # Find peaks which are higher than 0.5
    p = find_peaks(Lf, height=160)
      
    peaks = p[0]
    
    
    line_left   = None
    line_right  = None
    peaks_left  = peaks[peaks < CAMERA_CENTER]
    peaks_right = peaks[peaks > CAMERA_CENTER]
    
    # state machine
    
    if peaks_left.size:
        line_left = peaks_left.max()

    if peaks_right.size:
        line_right = peaks_right.min()
        
    if line_left and line_right:
        line_pos    = (line_left + line_right ) // 2
        track_width = line_right - line_left
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_STRAIGHT
        if(MOTOR_STRAIGHT >= SPEED_COMFY):
            MOTOR_STRAIGHT -= 20000
        MOTOR_TURN = MOTOR_TURN_INIT
        turn_ratio = turn_ratio_start
    elif line_left and not line_right:
#         track_width = 400
        line_pos    = line_left + int(track_width/turn_ratio)
#         line_pos    = line_left + int(track_width / 2)
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_TURN
        MOTOR_STRAIGHT = MOTOR_MAX
        if(MOTOR_TURN <= TURN_COMFY):
            MOTOR_TURN += 10000
        if(turn_ratio >= turn_ratio_end):
            turn_ratio -= inc
    elif not line_left and line_right:
#         track_width = 400
        line_pos    = line_right - int(track_width/turn_ratio)
#         line_pos    = line_right - int(track_width / 2)
        motor.duty_cycle = MOTOR_BRAKE + MOTOR_TURN
        MOTOR_STRAIGHT = MOTOR_MAX
        if(MOTOR_TURN <= TURN_COMFY):
            MOTOR_TURN += 10000
        if(turn_ratio >= turn_ratio_end):
            turn_ratio -= inc
    else:
        track_width = 400
        print("no line")
        turn_ratio = turn_ratio_start
        
    print(line_pos, peaks)

        
    DUTY_CYCLE = SERVO_MIDDLE + 5000 * (CAMERA_CENTER - line_pos)
    if DUTY_CYCLE > 2000000:
        DUTY_CYCLE = 2000000
    if DUTY_CYCLE < 1000000:
        DUTY_CYCLE = 1000000
        
    servo.duty_cycle =  DUTY_CYCLE
        
#Initialize lines position
#Check which lines are closer them in the next frame

        
        #print(line_pos)
        
motor.duty_cycle = MOTOR_BRAKE    
    

# Release resources
stream.close()
rawCapture.close()
camera.close()

291 [ 20 563 602]
291 [ 20 563 602]
291 [ 21 562 601]
292 [ 22 563 601]
296 [ 24 568 606]
299 [ 28 571 608]
304 [ 35 574 610]
311 [ 42 581]
319 [ 49 590]
323 [ 53 593 622]
327 [ 58 597 624]
329 [ 60 598 625]
328 [ 56 601 628]
325 [ 54 597 623]
322 [ 50 595 621]
317 [ 47 588 616]
314 [ 43 586 614]
311 [ 37 585 614]
309 [ 33 585 614]
306 [ 30 583 614]
304 [ 27 582 613]
306 [ 30 583 614]
305 [ 29 581 612]
308 [ 31 585 615]
309 [ 33 585 618]
310 [ 35 586 623]
311 [ 35 587 629]
312 [ 34 590 634]
314 [ 37 591]
314 [ 38 590]
314 [ 37 591]
312 [ 36 588]
311 [ 38 584]
319 [ 43 595 622]
330 [54]
360 [70]
389 [83]
423 [99]
458 [113]
488 [120]
494 [126]
502 [134]
503 [135]
506 [133 138]
513 [137 145]
516 [140 148]
389 [136 143 636]
381 [131 631]
365 [109 621]
348 [ 28  48  84 612]
329 [ 58 601]
311 [ 33 590]
296 [ 14 579 634]
283 [  1 566 619]
277 [559 606]
258 [555 595]
279 [  4 554 594]
286 [ 14 558 595]
299 [ 28 571 603]
313 [ 40 586 614]
323 [ 53 593 618]
333 [ 66 601 623]
341 [ 75 607 631]
34

In [32]:
# Release resources
stream.close()
rawCapture.close()
camera.close()
motor.duty_cycle = MOTOR_BRAKE   